<a href="https://colab.research.google.com/github/sabyasm/ipython-notebooks/blob/master/bert_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch


In [2]:
!rm -rf pytorch-pretrained-BERT
!git clone https://github.com/huggingface/pytorch-pretrained-BERT.git

Cloning into 'pytorch-pretrained-BERT'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1241 (delta 0), reused 1 (delta 0), pack-reused 1237
Receiving objects: 100% (1241/1241), 625.24 KiB | 5.17 MiB/s, done.
Resolving deltas: 100% (797/797), done.


In [12]:
!cp apex/ pytorch-pretrained-bert/

cp: -r not specified; omitting directory 'apex/'


In [0]:
#!git clone https://github.com/NVIDIA/apex.git
#!python apex/setup.py install  
#!pip install pytorch-pretrained-bert

In [7]:
import apex
import pytorch_pretrained_bert
pytorch_pretrained_bert.__version__

'0.4.0'

In [6]:
!mkdir dataset
!curl -Lo dataset/msr_paraphrase_train.txt https://s3-us-west-2.amazonaws.com/manu00/msr_paraphrase_train.txt
!curl -Lo dataset/msr_paraphrase_test.txt https://s3-us-west-2.amazonaws.com/manu00/msr_paraphrase_test.txt
!curl -Lo dataset/Competition_Train_Data.txt https://s3-us-west-2.amazonaws.com/manu00/Competition_Train_Data.txt
!curl -Lo dataset/Competition_Test_Data.txt https://s3-us-west-2.amazonaws.com/manu00/Competition_Test_Data.txt
!curl -Lo download_glue_data.py https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/becd574dd938f045ea5bd3cb77d1d506541b5345/download_glue_data.py

mkdir: cannot create directory ‘dataset’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1002k  100 1002k    0     0  1657k      0 --:--:-- --:--:-- --:--:-- 1654k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  422k  100  422k    0     0   809k      0 --:--:-- --:--:-- --:--:--  809k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1102k  100 1102k    0     0  1911k      0 --:--:-- --:--:-- --:--:-- 1914k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  274k  100  274k    0     0   506k      0 --:--:-- --:--:-- --:--:--  505

In [8]:
!python download_glue_data.py --data_dir='dataset' --tasks='MRPC' --path_to_mrpc=''

Processing MRPC...
	Completed!


In [0]:
!rm -rf /tmp/mrpc_output/

In [10]:
%%time

!python pytorch-pretrained-BERT/examples/run_classifier.py \
  --task_name MRPC \
  --do_train \
  --do_eval \
  --do_lower_case \
  --data_dir /content/dataset/MRPC/ \
  --bert_model bert-base-uncased \
  --max_seq_length 128 \
  --train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/mrpc_output/

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
12/17/2018 20:01:56 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
12/17/2018 20:01:56 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
12/17/2018 20:01:56 - INFO - __main__ -   LOOKING AT /content/dataset/MRPC/train.tsv
12/17/2018 20:01:56 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
12/1

In [13]:
#!ls -ltr /tmp/mrpc_output/
!cat /tmp/mrpc_output//eval_results.txt

eval_accuracy = 0.8357843137254902
eval_loss = 0.3809997632223017
global_step = 345
loss = 0.23087687298007634
